# Fetching observations from MET data base services

This notebook serves as sandbox for fetching data from frost.met, havvarsel-frost.met and thredds.met

See https://api.met.no/ for all interfaces and possible sources!!


## Havvarsel frost
Havvarsel frost delivers so far `temperature` measurements originated from badevann.no at seven beaches in the south of Norway (glider data integrated in the meantime, but not yet used here!). 
> Documentation:
> API documentation for obs/badevann https://havvarsel-frost.met.no/docs/apiref#/obs%2Fbadevann/obsBadevannGet 
> Datastructure described on https://havvarsel-frost.met.no/docs/dataset_badevann


## Thredds
Holds netcdf files with the a bunch of different data

> See the catalog: https://thredds.met.no/thredds/catalog.html

We primarily use the `Ocean and Ice/met.no (OLD) ROMS NorKyst800m coastal forecasting system` to get the forecasted water temperatures


In [1]:
# Importing general libraries
import sys
import json
import datetime
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import netCDF4
import pyproj as proj


In [2]:
start_time = datetime.datetime.strptime("2021-10-10T06:00", "%Y-%m-%dT%H:%M")
end_time = datetime.datetime.strptime("2021-10-12T12:00", "%Y-%m-%dT%H:%M")


In [3]:
import HavvarselFrostImporter
havvarsel = HavvarselFrostImporter.HavvarselFrostImporter(start_time,end_time)
df_loc, df = havvarsel.data(100)

Trying https://havvarsel-frost.met.no/api/v1/obs/badevann/get?time=2021-10-10T06:00:00Z/2021-10-12T12:00:00Z&incobs=true&buoyids=100&parameter=temperature
  buoyid    parameter     name        lon        lat
0    100  temperature  Sørenga  10.749700  59.900700


In [4]:
lon = float(df_loc["lon"])
lat = float(df_loc["lat"])
start_time = df.index[0].to_pydatetime().replace(tzinfo=None)
end_time = df.index[-1].to_pydatetime().replace(tzinfo=None)

In [5]:
import NorKystImporter
norkyst = NorKystImporter.NorKystImporter(start_time, end_time)

In [6]:
norkyst_df = norkyst.norkyst_data("temperature",lon,lat)

Processing  https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m-1h/NorKyst-800m_ZDEPTHS_his.an.2021101000.nc
Coordinates model (x,y= 544,180): 59.898461480861215, 10.740797814175055
Processing  https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m-1h/NorKyst-800m_ZDEPTHS_his.an.2021101100.nc
Processing  https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m-1h/NorKyst-800m_ZDEPTHS_his.an.2021101200.nc


In [10]:
from importlib import reload

In [11]:
reload(PPImporter)
pp = PPImporter.PPImporter(start_time, end_time)

In [12]:
pp_df = pp.pp_data(["air_temperature_2m"],lon,lat)

Filename timestamp based on start_time: 2021101006
Filename timestamp based on end_time: 2021101211
Processing https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T06Z.nc
Coordinates model (x,y= 660,767): 59.89720311267746, 10.745418590272147
Processing  https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T06Z.nc
Processing  https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T07Z.nc
Processing  https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T08Z.nc
Processing  https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T09Z.nc
Processing  https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T10Z.nc
Processing  https://thredds.met.no/thredds/dodsC/metpparchive/2021/10/10/met_analysis_1_0km_nordic_20211010T11Z.nc
Processing  https://thre

In [13]:
pp_df

,air_temperature_2m
referenceTime,
2021-10-10 06:00:00+00:00,284.421204
2021-10-10 07:00:00+00:00,284.653046
2021-10-10 08:00:00+00:00,284.960754
2021-10-10 09:00:00+00:00,285.314545
2021-10-10 10:00:00+00:00,285.584534
...,...
2021-10-14 00:00:00+00:00,280.768097
2021-10-14 01:00:00+00:00,280.676086
2021-10-14 02:00:00+00:00,280.535980
